In [1]:
class Test_Semitic:
  @staticmethod
  def get_stem_df(x):
    try:
      print("checked")
      return get_cnn_stem(x)
    except:
      print("error")
      return "error"

  @staticmethod
  def test_full_equality(row):
    return 1 if row['predicted_root'] == row['root'] else 0

  @staticmethod
  def test_first(row):
    return 1 if row['first_root_letter'] == row['first_pred_letter'] else 0

  @staticmethod
  def test_second(row):
    return 1 if row['second_root_letter'] == row['second_pred_letter'] else 0

  @staticmethod
  def test_third(row):
    return 1 if row['third_root_letter'] == row['third_pred_letter'] else 0


  @staticmethod
  def at_least_one_not_in_word(row):
    '''
    Returns 1 if the word has at least one letter that is not in the word
    '''
    root = row['root']
    word = row['word']
    sum = 0
    for i in range(len(root)):
      if root[i] in word:
        sum += 1
    return 0 if sum == 3 else 1

In [2]:
import numpy as np
import pandas as pd

vocabulary = ['ء', 'آ', 'أ', 'ؤ', 'إ', 'ئ', 'ا', 'ب', 'ة', 'ت', 'ث', 'ج', 'ح', 'خ',
                      'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك',
                      'ل', 'م', 'ن', 'ه', 'و', 'ى', 'ي']

max_len = 8

def string_vectorizer(string, vocabulary, max_len) -> np.array:
  '''
  Returns a matrix representation of the string
  '''
  empty = np.zeros((max_len, len(vocabulary)))
  for i,l in enumerate(string):
    empty[i, vocabulary.index(l)] = 1
  matrix_representation = empty
  return matrix_representation

def get_X_for_prediction(word):
  test = pd.DataFrame({'words': [word]})
  test['word_as_matrix'] = test['words'].apply(lambda x: string_vectorizer(x, vocabulary, 8))
  X_test_real = np.stack(test['word_as_matrix'].to_numpy())
  return X_test_real

def get_cnn_stem(word):
  X = get_X_for_prediction(word)
  pred = ''
  for i in range(3):
    pred += vocabulary[np.argmax(cnn_model.predict(X, verbose = 0)[0, i])]
  return pred

def decode_X(X):
  decoded_word = ''
  for i in range(X.shape[0]):
    tmp_letter = vocabulary[np.argmax(X[i])]
    decoded_word+=tmp_letter
  return decoded_word.strip()

def get_stem_from_X(X):
  decoded = decode_X(X)
  return predict_stem(decoded)

def test(row):
  return 1 if row['true'] == row['pred'] else 0

In [3]:
!git clone https://github.com/delmedigo88/Semitic.git

fatal: destination path 'Semitic' already exists and is not an empty directory.


In [4]:
# load cnn model
import pickle
import pandas as pd
cnn_model_path = '/content/Semitic/trained_models/CNN_model_28_3_24/cnn_full_char_2024-03-28 11_15.pkl'
with open(cnn_model_path , 'rb') as f:
    cnn_model = pickle.load(f)

In [11]:
test_df = pd.read_excel('/content/Semitic/data/test_data/word-root-table_side.xlsx').sample(10000)

In [12]:
def get_stem_df(x):
  try:
    print("checked")
    return get_cnn_stem(x)
  except:
    print("error")
    return "error"
#list comp
test_df['predicted_root'] = [get_stem_df(x) for x in test_df['word']]


def test(row):
  return 1 if row['predicted_root'] == row['root'] else 0

test_df['check'] =  test_df.apply(test, axis=1)
test_df.check.mean()

Streaming output truncated to the last 5000 lines.
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
checked
check

0.9609

In [13]:
df_metrics = test_df[['word','root','predicted_root','check']].copy()
df_metrics['first_root_letter'] = df_metrics['root'].apply(lambda x: x[0])
df_metrics['first_pred_letter'] = df_metrics['predicted_root'].apply(lambda x: x[0])

df_metrics['second_root_letter'] = df_metrics['root'].apply(lambda x: x[1])
df_metrics['second_pred_letter'] = df_metrics['predicted_root'].apply(lambda x: x[1])

df_metrics['third_root_letter'] = df_metrics['root'].apply(lambda x: x[2])
df_metrics['third_pred_letter'] = df_metrics['predicted_root'].apply(lambda x: x[2])

df_metrics['t1'] = df_metrics.apply(lambda x: Test_Semitic.test_first(x), axis=1)
df_metrics['t2'] = df_metrics.apply(lambda x: Test_Semitic.test_second(x), axis=1)
df_metrics['t3'] = df_metrics.apply(lambda x: Test_Semitic.test_third(x), axis=1)

df_metrics[['word','root','predicted_root','t1', 't2', 't3']] ## pairwise accurcy
df_metrics['pairwise_acc'] = (df_metrics['t1'] + df_metrics['t2'] + df_metrics['t3']) / 3
df_metrics['correct'] = df_metrics['t1'] + df_metrics['t2'] + df_metrics['t3']

df_metrics['at_least_one'] = df_metrics.apply(lambda x: Test_Semitic.at_least_one_not_in_word(x), axis=1)

In [15]:
# Build Testing Report:

from datetime import datetime

now = datetime.now()

current_date = now.strftime("%Y-%m-%d")
current_hour = now.hour
current_minute = now.minute

test_summary = f"Report Time: {current_date} {current_hour:02d}:{current_minute:02d}\n"
test_summary += f"Number of words: {len(test_df)}\n"
test_summary += f"Number of correct words: {test_df.check.sum()}\n"
test_summary += f"Number of incorrect words: {test_df.check.count() - test_df.check.sum()}\n"
test_summary += f"Accuracy: {test_df.check.mean():.2%}\n"
test_summary += f"Pairwise Accuracy: {df_metrics['pairwise_acc'].mean():.2%}\n"
test_summary += f"0 letters accuracy: {df_metrics.correct.value_counts()[3]}\n"
test_summary += f"1 letters accuracy: {df_metrics.correct.value_counts()[2]}\n"
test_summary += f"2 letters accuracy: {df_metrics.correct.value_counts()[1]}\n"
test_summary += f"3 letters accuracy: {df_metrics.correct.value_counts()[0]}\n"
test_summary += f"Only some root letteres in word percent: {df_metrics.query('at_least_one == 1').count()[0] / len(test_df):.2%}\n"
test_summary += f"Only some root letteres in word accuracy: {df_metrics.groupby('at_least_one').check.mean()[1]:.2%}\n"
test_summary += f"All root letters in word accuracy: {df_metrics.groupby('at_least_one').check.mean()[0]:.2%}\n"
print(test_summary)

with open(f'test_summary{current_date} {current_hour:02d}:{current_minute:02d}.txt', 'w') as f:
    f.write(test_summary)

df_metrics.to_excel(f'test_summary{current_date} {current_hour:02d}:{current_minute:02d}.xlsx')

# download files

from google.colab import files
files.download(f'test_summary{current_date} {current_hour:02d}:{current_minute:02d}.xlsx')
files.download(f'test_summary{current_date} {current_hour:02d}:{current_minute:02d}.txt')

Report Time: 2024-03-28 12:17
Number of words: 10000
Number of correct words: 9609
Number of incorrect words: 391
Accuracy: 96.09%
Pairwise Accuracy: 98.31%
0 letters accuracy: 9609
1 letters accuracy: 296
2 letters accuracy: 74
3 letters accuracy: 21
Only some root letteres in word percent: 18.55%
Only some root letteres in word accuracy: 90.73%
All root letters in word accuracy: 97.31%



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
cnn_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 6, 34, 32)         320       
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 6, 17, 32)         0         
 ng2D)                                                           
                                                                 
 conv2d_19 (Conv2D)          (None, 4, 15, 64)         18496     
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 4, 7, 64)          0         
 ng2D)                                                           
                                                                 
 flatten_9 (Flatten)         (None, 1792)              0         
                                                                 
 dense_26 (Dense)            (None, 512)              